In [1]:
import cv2
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd
import numpy as np
from PIL import Image
import io
import os
import os.path
import glob
from datetime import datetime
from google.cloud import vision
from google.cloud.vision import types

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bin/ROKS-78e082836307.json"

plt.figure(figsize=(50, 40))
plt.style.use('dark_background')

images = glob.glob('data/info/original/*.jpg')
print(images.index('data/info/original/Screenshot_20200727-203539_Rise of Kingdoms.jpg'))

ValueError: 'data/info/original/Screenshot_20200727-203539_Rise of Kingdoms.jpg' is not in list

<Figure size 3600x2880 with 0 Axes>

In [3]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    # [START vision_python_migration_text_detection]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in texts[0].bounding_poly.vertices])

    return texts[0].description

In [4]:
def change_to_number(s_power):
    list_of_numbers = re.findall(r'\d+', s_power)
    return int(''.join(list_of_numbers))

In [5]:
def change_to_list(chars):
    tmp_texts = list()
    s = ""
    for c in chars:
        if c != '\n':
            s += c
        else:
            tmp_texts.append(s)
            s = ""
    
    return tmp_texts

In [125]:
#crop & convert
start_time = datetime.now()

for image in images:
    initial = Image.open(image)
    width = initial.size[0]
    height = initial.size[1]

    path, file_name = os.path.split(image)

    #name
    crop_name = initial.crop((width*0.295, height*0.15, width*0.8, height*0.2))

    #kill
    crop_kill = initial.crop((width*0.56, height*0.28, width*0.72, height*0.44))
    img_white = Image.new('RGB', (57, 231), (255, 255, 255, 255))
    crop_kill.paste(img_white, (0, 0, 57, 231))
    crop_kill.paste(img_white, (243, 0, 300, 231))

    img = np.array(crop_kill) 
    img = img[:, :, ::-1].copy() 

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
    gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    ret, img_thresh = cv2.threshold(
    img_blurred, 
    140,
    255,
    cv2.THRESH_BINARY_INV
    )
    crop_kill = Image.fromarray(img_thresh)

    crop_ds = initial.crop((width*0.20, height*0.23, width*0.8, height*0.98))

    nw, nh = crop_name.size
    kw, kh = crop_kill.size
    dsw, dsh = crop_ds.size

    crop_name.save('data/info/croped/name/' + file_name)
    crop_kill.save('data/info/croped/kill/' + file_name)
    crop_ds.save('data/info/croped/ds/' + file_name)

    merge = Image.new('RGB', (bw, nh+kh+dsh),(0, 0, 0))

    merge.paste(crop_name, (0, 0, nw, nh))
    merge.paste(crop_kill, (0, nh, kw, nh+kh))
    merge.paste(crop_ds, (0, nh+kh, dsw, th+kh+dsh))

    merge.save('data/info/convert/' + file_name)

end_time = datetime.now()

print (end_time - start_time)

0:00:41.441502


In [127]:
start_time = datetime.now()

name = list()
power = list()
kill = list()
dead = list()
support = list()
t1 = list()
t2 = list()
t3 = list()
t4 = list()
t5 = list()
i = 0
for image in images:
    path, file_name = os.path.split(image)
    text = detect_text('data/info/convert/' + file_name)
    texts = change_to_list(text)
    
    name.append(texts[0])
    p = texts[1]
    pp = p[p.index(':')+1:-1]
    power.append(change_to_number(pp))

    k = texts[2]
    kk = k[k.index(':')+1:-1]
    kill.append(change_to_number(kk))
    t1.append(change_to_number(texts[3]))
    t2.append(change_to_number(texts[4]))
    t3.append(change_to_number(texts[5]))
    t4.append(change_to_number(texts[6]))
    t5.append(change_to_number(texts[7]))
    dead.append(change_to_number(texts[texts.index('전사')+1]))
    support.append(change_to_number(texts[texts.index('원조')+1]))
    
    print(i, 'done...')
    i += 1
    
end_time = datetime.now()

print (end_time - start_time)

df = pd.DataFrame({'name': name,
                   'power':power,
                   'kill':kill,
                   'T1_kill':t1,
                   'T2_kill':t2,
                   'T3_kill':t3,
                   'T4_kill':t4,
                   'T5_kill':t5,
                   'dead':dead,
                   'support':support})

df.to_excel('data/info/result6.xlsx')

0 done...
1 done...
2 done...
3 done...
4 done...
5 done...
6 done...
7 done...
8 done...
9 done...
10 done...
11 done...
12 done...
13 done...
14 done...
15 done...
16 done...
17 done...
18 done...
19 done...
20 done...
21 done...
22 done...
23 done...
24 done...
25 done...
26 done...
27 done...
28 done...
29 done...
30 done...
31 done...
32 done...
33 done...
34 done...
35 done...
36 done...
37 done...
38 done...
39 done...
40 done...
41 done...
42 done...
43 done...
44 done...
45 done...
46 done...
47 done...
48 done...
49 done...
50 done...
51 done...
52 done...
53 done...
54 done...
55 done...
56 done...
57 done...
58 done...
59 done...
60 done...
61 done...
62 done...
63 done...
64 done...
65 done...
66 done...
67 done...
68 done...
69 done...
70 done...
71 done...
72 done...
73 done...
74 done...
75 done...
76 done...
77 done...
78 done...
79 done...
80 done...
81 done...
82 done...
83 done...
84 done...
85 done...
86 done...
87 done...
88 done...
89 done...
90 done...
91 done..

KeyboardInterrupt: 

In [120]:
#crop & convert
start_time = datetime.now()

initial = Image.open(images[39])
width = initial.size[0]
height = initial.size[1]

path, file_name = os.path.split(images[39])

#name
crop_name = initial.crop((width*0.295, height*0.15, width*0.8, height*0.2))

#kill
crop_kill = initial.crop((width*0.56, height*0.28, width*0.72, height*0.44))
img_white = Image.new('RGB', (57, 231), (255, 255, 255, 255))
crop_kill.paste(img_white, (0, 0, 57, 231))
crop_kill.paste(img_white, (243, 0, 300, 231))

img = np.array(crop_kill) 
img = img[:, :, ::-1].copy() 

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

ret, img_thresh = cv2.threshold(
img_blurred, 
140,
255,
cv2.THRESH_BINARY_INV
)
crop_kill = Image.fromarray(img_thresh)

crop_ds = initial.crop((width*0.20, height*0.23, width*0.8, height*0.98))

nw, nh = crop_name.size
kw, kh = crop_kill.size
dsw, dsh = crop_ds.size

crop_name.save('data/info/croped/name/' + file_name)
crop_kill.save('data/info/croped/kill/' + file_name)
crop_ds.save('data/info/croped/ds/' + file_name)

merge = Image.new('RGB', (bw, nh+kh+dsh),(0, 0, 0))

merge.paste(crop_name, (0, 0, nw, nh))
merge.paste(crop_kill, (0, nh, kw, nh+kh))
merge.paste(crop_ds, (0, nh+kh, dsw, th+kh+dsh))

merge.save('data/info/convert/' + file_name)

end_time = datetime.now()

print (end_time - start_time)

0:00:00.243485


In [123]:
start_time = datetime.now()

name = list()
power = list()
kill = list()
dead = list()
support = list()
t1 = list()
t2 = list()
t3 = list()
t4 = list()
t5 = list()
i = 0
# for image in images:
path, file_name = os.path.split(images[39])
text = detect_text('data/info/convert/' + file_name)
texts = change_to_list(text)

s[s.index(':')+1:-1]
name.append(texts[0])

p = texts[1]
pp = p[p.index(':')+1:-1]
power.append(change_to_number(pp))

k = texts[2]
kk = k[k.index(':')+1:-1]
kill.append(change_to_number(kk))
t1.append(change_to_number(texts[3]))
t2.append(change_to_number(texts[4]))
t3.append(change_to_number(texts[5]))
t4.append(change_to_number(texts[6]))
t5.append(change_to_number(texts[7]))
dead.append(change_to_number(texts[texts.index('전사')+1]))
support.append(change_to_number(texts[texts.index('원조')+1]))

print(i, 'done...')
i += 1
    
end_time = datetime.now()

print (end_time - start_time)

0 done...
0:00:02.414502


In [124]:
# print(texts[0])
# print(texts[1])
# s = texts[2]
# print(texts[2])
# print(texts[3])
# print(texts[4])
# print(texts[5])
# print(texts[6])
# print(texts[7])
# print(texts[texts.index('전사')+1])
# print(texts[texts.index('원조')+1])
print(name[0])
print(power[0])
print(kill[0])
print(t1[0])
print(t2[0])
print(t3[0])
print(t4[0])
print(t5[0])
print(dead[0])
print(support[0])


타 로
5421077
1665902
5187582
3757254
2169900
5280191
264098
1193265
307945382


In [78]:
print(images.index('data/info/original/Screenshot_20200727-204403_Rise of Kingdoms.jpg'))
print(images.index('data/info/original/Screenshot_20200727-203452_Rise of Kingdoms.jpg'))
print(images.index('data/info/original/Screenshot_20200727-203422_Rise of Kingdoms.jpg'))
#126

54
83
39


In [79]:
path, file_name = os.path.split(images[54])
text = detect_text('data/info/convert/' + file_name)
texts = change_to_list(text)
print(texts)

['탱글경리', '전투력: 34,296,966', '*1: 3,495,867', '268,690', '648,927', '1,047,416', '1,096,569', '159,407', '전투 통계', '처치 정보: (2019/10/23 후부터 계산 시작)', '역대 최고 전력', '268,690', 'O 648,92734,301,742', 'O 1,047,416', 'O 1,096,569', '승리', '14,373', '159,407', '패배', '133', '전사', '917,367', '정찰 횟수', '1,810', '자원 통계', '채집한 자원', '4,763,217,757', '원조', '349,811,063', '연맹 지원 횟수', '11,650']


In [92]:
s

'*/2: 38,586,811'

In [112]:
ss = s[s.index(':')+1:-1]

In [113]:
ss

' 38,586,81'